<a href="https://colab.research.google.com/github/Puneet-Bajaj-IITM/01245/blob/main/Chat_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl https://ollama.ai/install.sh | sh
!nohup ollama serve &
!pip install ollama
!ollama pull llama2
!pip install flask-ngrok
!pip install pyngrok

import pandas as pd
import concurrent.futures

In [13]:
import os

templates_folder = '/content/templates'
if not os.path.exists(templates_folder):
    os.makedirs(templates_folder)

index_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>File Upload and Processing</title>
    <!-- Bootstrap CSS -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha1/dist/css/bootstrap.min.css" rel="stylesheet">
    <!-- Tailwind CSS -->
    <link href="https://cdn.jsdelivr.net/npm/tailwindcss@3.0.7/dist/tailwind.min.css" rel="stylesheet">
    <!-- Custom CSS -->
    <style>
        @keyframes roll {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        .rolling-animation {
            animation: roll 1s linear infinite;
        }
    </style>
</head>
<body class="bg-gray-100 p-4">
    <div class="container mx-auto">
        <h1 class="text-3xl font-bold mb-4">Upload Excel Files</h1>
        <form action="/upload" method="post" enctype="multipart/form-data" class="mb-4">
            <div class="mb-3">
                <label for="files" class="form-label">Choose Excel Files:</label>
                <input type="file" class="form-control" id="files" name="files[]" multiple accept=".xlsx" onchange="displaySelectedFiles(this)">
            </div>
            <button type="submit" class="btn btn-primary">Upload</button>
        </form>
        <h2 class="text-2xl font-bold mb-2">Selected Files:</h2>
        <ul id="selectedFilesList" class="list-disc pl-5">
            {% for file in selected_files %}
            <li>{{ file }}</li>
            {% endfor %}
        </ul>
        {% if output_files %}
        <h2 class="text-2xl font-bold mb-2">Processed Files:</h2>
        <ul class="list-disc pl-5">
            {% for file in output_files %}
                <li><a href="/download/{{ file }}" class="text-blue-500">{{ file }}</a></li>
            {% endfor %}
        </ul>
        {% endif %}
    </div>
    <script>
        function displaySelectedFiles(input) {
            const fileList = document.getElementById('selectedFilesList');
            fileList.innerHTML = ''; // Clear previous list

            const files = input.files;
            for (let i = 0; i < files.length; i++) {
                const listItem = document.createElement('li');
                listItem.textContent = files[i].name;
                fileList.appendChild(listItem);
            }
        }
    </script>
</body>
</html>

"""

with open(os.path.join(templates_folder, 'index.html'), 'w') as f:
    f.write(index_content)

print("Templates folder and index.html created successfully.")

Templates folder and index.html created successfully.


In [14]:

def generate_summary(row):
    if row['Chat']:
        res = ollama.generate(model='llama2', prompt=f"""Summarize {row['Chat']}, mention the main issues encountered by the customer, the solutions provided by agent and overarching themes discussed""")
    return res['response'] or None

def process_row(row):
    summary = generate_summary(row)
    return summary

def process_file(filename):
    data = pd.read_excel(filename, header=None)
    data['Summary'] = None
    data.columns = ['Chat', 'Summary']

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_row, row) for _, row in data.iterrows()]
        for future, (index, _) in zip(futures, data.iterrows()):
            summary = future.result()
            data.at[index, 'Summary'] = summary
    return data


In [15]:
from flask import Flask, request, render_template, send_file
from pyngrok import ngrok
import ollama
import os
import pandas as pd
ngrok.set_auth_token("2aGUSiqfIzgKx0yh6XHaTX3G11f_5HRDp7atGGPtyxaiH91bc")

app = Flask(__name__)

UPLOAD_FOLDER = '/content/uploads'
OUTPUT_FOLDER = '/content/outputs'
ALLOWED_EXTENSIONS = {'xlsx'}

if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['OUTPUT_FOLDER'] = OUTPUT_FOLDER

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def index():
    output_files = os.listdir(app.config['OUTPUT_FOLDER'])
    return render_template('index.html', output_files=output_files)

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'files[]' not in request.files:
        return 'No file part'

    files = request.files.getlist('files[]')
    for file in files:
        if file and allowed_file(file.filename):
            filename = file.filename
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            processed_df = process_file(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            output_filename = filename.replace('.xlsx', '_processed.xlsx')
            output_path = os.path.join(app.config['OUTPUT_FOLDER'], output_filename)
            processed_df.to_excel(output_path, index=False)

    output_files = os.listdir(app.config['OUTPUT_FOLDER'])
    return render_template('index.html', output_files=output_files)

@app.route('/download/<filename>')
def download_file(filename):
    return send_file(os.path.join(app.config['OUTPUT_FOLDER'], filename), as_attachment=True)


if __name__ == '__main__':
    from pyngrok import ngrok

    public_url = ngrok.connect()
    print(' * Tunnel URL:', public_url)

    app.run(port=80)

